<a href="https://colab.research.google.com/github/elliekim9881/AIFFELthon_I15/blob/main/keywords/keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!pip install konlpy

In [62]:
import networkx as nx
from itertools import combinations
from konlpy.tag import Okt
import re
import pandas as pd

In [63]:
!pip install xlrd

In [64]:
file_path = '/content/1_30000_20231101.xls'
df = pd.read_excel(file_path)
df.head()

,표제어,동형어 번호,구분,품사,고유어 여부,원어,발음,활용,파생어,☞(가 보라),...,러시아어 부표제어 대역어,러시아어 부표제어 대역어 뜻풀이,영어 부표제어 대역어,영어 부표제어 대역어 뜻풀이,일본어 부표제어 대역어,일본어 부표제어 대역어 뜻풀이,프랑스어 부표제어 대역어,프랑스어 부표제어 대역어 뜻풀이,스페인어 부표제어 대역어,스페인어 부표제어 대역어 뜻풀이
0,가,1,단어,명사,고유어,NaN,가ː,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,가,1,단어,명사,고유어,NaN,가ː,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-가,1,단어,접사,한자어,哥,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,가-,1,단어,접사,한자어,假,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,가,2,단어,명사,한자어,可,가ː,NaN,가하다2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df = df[['표제어', '의미 범주', '뜻풀이', '용례']]
df.head()

,표제어,의미 범주,뜻풀이,용례
0,가,자연 > 지형,어떤 장소나 물건의 둘레나 끝부분.,<구> 가를 꾸미다.\n<구> 가를 장식하다.\n<구> 가에 걸치다.\n<구> 가에...
1,가,자연 > 지형,‘주변’의 뜻을 나타내는 말.,<구> 강가.\n<구> 길가.\n<구> 냇가.\n<구> 문가.\n<구> 시냇가.\n...
2,-가,NaN,‘그 성을 가진 사람’의 뜻을 더하는 접미사.,<구> 김가\n<구> 박가\n<구> 이가\n\n
3,가-,NaN,‘가짜의’ 또는 ‘임시의’의 뜻을 더하는 접두사.,<구> 가건물\n<구> 가계약\n<구> 가등기\n<구> 가매장\n<구> 가석방\n\n
4,가,NaN,회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.,<구> 가와 부.\n<구> 가를 밝히다.\n<문장> 곧 표결을 시작하오니 가인지 부...


불용어 제거

In [66]:
pip install nltk

In [67]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
from nltk.tokenize import word_tokenize

# 1. txt 파일에서 불용어 목록을 읽어오기
with open('/content/korean_stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read().splitlines()

# 2. 읽어온 불용어 목록을 리스트로 변환

# '뜻풀이', '용례' 열의 각 문장에서 불용어를 제거하는 함수
def remove_stopwords(text):
    if isinstance(text, str):  # text가 문자열인 경우에만 처리
        word_tokens = word_tokenize(text)
        result = [w for w in word_tokens if w not in stop_words]
        return " ".join(result)
    else:
        return text  # NaN 또는 비어 있는 문자열인 경우 그대로 반환

# '뜻풀이' , '용례' 열에 불용어 처리 적용
df['뜻풀이'] = df['뜻풀이'].apply(remove_stopwords)
df['용례'] = df['용례'].apply(remove_stopwords)

# 결과 출력
df.head()

,표제어,의미 범주,뜻풀이,용례
0,가,자연 > 지형,장소나 물건의 둘레나 끝부분,가를 꾸미다 가를 장식하다 가에 걸치다 가에 달다 가에 달라붙다 가에 세우다 가에 ...
1,가,자연 > 지형,주변 뜻을 나타내는 말,강가 길가 냇가 문가 시냇가 우물가 창가 창문가 호숫가
2,-가,NaN,성을 가진 사람 뜻을 더하는 접미사,김가 박가 이가
3,가-,NaN,가짜의 또는 임시의 뜻을 더하는 접두사,가건물 가계약 가등기 가매장 가석방
4,가,NaN,회의 등에서 안건에 대해 표결을 할 찬성한다는 표시,가와 부 가를 밝히다 문장 표결을 시작하오니 가인지 부인지 여부를 정해 주시기 바랍...


In [69]:
from konlpy.tag import Okt
okt = Okt()

In [70]:
# '뜻풀이' 열의 주요 단어 및 핵심 단어 추출 및 '정답 후보' 열 추가
def extract_keywords(text):
    if isinstance(text, str):
        # 형태소 분석
        morphemes = okt.pos(text)
        # 명사 및 핵심 단어 추출
        keywords = [word for word, pos in morphemes if pos in ['Noun', 'Adjective', 'Verb', 'Adverb']]
        # 중복을 제거하여 리스트로 변환
        unique_keywords = list(set(keywords))
        return ' '.join(unique_keywords)
    else:
        return text  # NaN 또는 비어 있는 문자열인 경우 그대로 반환

In [71]:
# '뜻풀이' 열에 주요 및 핵심 단어 추출 함수 적용
df['정답 후보'] = df['뜻풀이'].apply(extract_keywords)

# 결과 출력
df.head()

,표제어,의미 범주,뜻풀이,용례,정답 후보
0,가,자연 > 지형,장소나 물건의 둘레나 끝부분,가를 꾸미다 가를 장식하다 가에 걸치다 가에 달다 가에 달라붙다 가에 세우다 가에 ...,끝 장소 둘레 물건 부분
1,가,자연 > 지형,주변 뜻을 나타내는 말,강가 길가 냇가 문가 시냇가 우물가 창가 창문가 호숫가,주변 뜻 말 나타내는
2,-가,NaN,성을 가진 사람 뜻을 더하는 접미사,김가 박가 이가,성 사람 뜻 가진 접미사 더하는
3,가-,NaN,가짜의 또는 임시의 뜻을 더하는 접두사,가건물 가계약 가등기 가매장 가석방,또는 가짜 임시 뜻 접두사 더하는
4,가,NaN,회의 등에서 안건에 대해 표결을 할 찬성한다는 표시,가와 부 가를 밝히다 문장 표결을 시작하오니 가인지 부인지 여부를 정해 주시기 바랍...,할 찬성 회의 안건 등 표결 대해 표시


In [74]:
# '정답 후보' 열에서 가장 대표적인 명사 선택 함수
def select_representative_noun(text):
    if isinstance(text, str) and text:  # 비어 있지 않은 문자열만 처리
        # 형태소 분석을 통해 명사 추출
        morphemes = okt.pos(text)
        nouns = [word for word, pos in morphemes if pos == 'Noun']
        if nouns:  # 명사가 있는 경우에만 가장 높은 빈도수를 가진 명사 선택
            word_counts = {}
            for word in nouns:
                if word in word_counts:
                    word_counts[word] += 1
                else:
                    word_counts[word] = 1
            main_noun = max(word_counts, key=word_counts.get)
            return main_noun
    return text  # NaN 또는 빈 문자열인 경우 그대로 반환

In [75]:
# '정답' 열에 가장 대표적인 명사 선택 함수 적용
df['정답'] = df['정답 후보'].apply(select_representative_noun)

# 결과 출력
df.head()

,표제어,의미 범주,뜻풀이,용례,정답 후보,주요 단어,정답
0,가,자연 > 지형,장소나 물건의 둘레나 끝부분,가를 꾸미다 가를 장식하다 가에 걸치다 가에 달다 가에 달라붙다 가에 세우다 가에 ...,끝 장소 둘레 물건 부분,끝,끝
1,가,자연 > 지형,주변 뜻을 나타내는 말,강가 길가 냇가 문가 시냇가 우물가 창가 창문가 호숫가,주변 뜻 말 나타내는,주변,주변
2,-가,NaN,성을 가진 사람 뜻을 더하는 접미사,김가 박가 이가,성 사람 뜻 가진 접미사 더하는,성,성
3,가-,NaN,가짜의 또는 임시의 뜻을 더하는 접두사,가건물 가계약 가등기 가매장 가석방,또는 가짜 임시 뜻 접두사 더하는,또는,가짜
4,가,NaN,회의 등에서 안건에 대해 표결을 할 찬성한다는 표시,가와 부 가를 밝히다 문장 표결을 시작하오니 가인지 부인지 여부를 정해 주시기 바랍...,할 찬성 회의 안건 등 표결 대해 표시,할,찬성
